In [46]:

import numpy as np 
import pandas as pd 

import json
from pandas.io.json import json_normalize
import seaborn as sns
import matplotlib.pyplot as plt
import lightgbm as lgb
from sklearn.metrics import explained_variance_score, roc_auc_score, precision_recall_curve, roc_curve, average_precision_score,accuracy_score
from sklearn import model_selection, preprocessing, metrics
from sklearn.model_selection import TimeSeriesSplit, GridSearchCV, cross_val_score, cross_validate
from scipy import stats
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.linear_model import LinearRegression
import plotly.plotly as py #For World Map
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

from mpl_toolkits.basemap import Basemap

import gc
import os
import csv
print(os.listdir("."))

['.DS_Store', '.ipynb_checkpoints', 'citations.ipynb', 'data']


In [34]:
df = pd.read_json('./data/dblp-ref/dblp-ref-0.json', lines=True)
df.append(pd.read_json('./data/dblp-ref/dblp-ref-1.json', lines=True))
df.append(pd.read_json('./data/dblp-ref/dblp-ref-2.json', lines=True))
df.append(pd.read_json('./data/dblp-ref/dblp-ref-3.json', lines=True))
    

,abstract,authors,id,n_citation,references,title,venue,year
0,The purpose of this study is to develop a lear...,"[Makoto Satoh, Ryo Muramatsu, Mizue Kayama, Ka...",00127ee2-cb05-48ce-bc49-9de556b93346,0,"[51c7e02e-f5ed-431a-8cf5-f761f266d4be, 69b625b...",Preliminary Design of a Network Protocol Learn...,international conference on human-computer int...,2013
1,This paper describes the design and implementa...,"[Gareth Beale, Graeme Earl]",001c58d3-26ad-46b3-ab3a-c1e557d16821,50,"[10482dd3-4642-4193-842f-85f3b70fcf65, 3133714...",A methodology for the physically accurate visu...,visual analytics science and technology,2011
2,This article applied GARCH model instead AR or...,"[Altaf Hossain, Faisal Zaman, Mohammed Nasser,...",001c8744-73c4-4b04-9364-22d31a10dbf1,50,"[2d84c0f2-e656-4ce7-b018-90eda1c132fe, a083a1b...","Comparison of GARCH, Neural Network and Suppor...",pattern recognition and machine intelligence,2009
3,NaN,"[Jea-Bum Park, Byungmok Kim, Jian Shen, Sun-Yo...",00338203-9eb3-40c5-9f31-cbac73a519ec,0,"[8c78e4b0-632b-4293-b491-85b1976675e6, 9cdc54f...",Development of Remote Monitoring and Control D...,,2011
4,NaN,"[Giovanna Guerrini, Isabella Merlo]",0040b022-1472-4f70-a753-74832df65266,2,NaN,Reasonig about Set-Oriented Methods in Object ...,,1998
5,NaN,"[Rafael Álvarez, Leandro Tortosa, José-Francis...",005ce28f-ed77-4e97-afdc-a296137186a1,0,NaN,COMPARING GNG3D AND QUADRIC ERROR METRICS METH...,international conference on computer graphics ...,2009
6,NaN,"[Jovan Dj. Golic, Guglielmo Morgari]",00638a94-23bf-4fa6-b5ce-40d799c65da7,2,NaN,Vectorial fast correlation attacks.,,2004
7,NaN,"[Guzin Ulutas, Mustafa Ulutas, Vasif V. Nabiyev]",00701b05-684f-45f9-b281-425abfec482c,0,"[5626736c-e434-4e2d-8405-54940fab88ab, 8e87e87...",Improved Secret Image Sharing Method By Encodi...,international symposium on computer and inform...,2011
8,NaN,"[Pranay Chaudhuri, Hussein Thompson]",00745041-3636-4d18-bbec-783c4278c40d,0,NaN,A Self-Stabilizing Algorithm for Finding the C...,parallel and distributed processing techniques...,2003
9,NaN,"[Dominik Szajerman, Adam Jurczyński]",00964544-cbe2-4da9-bb5a-03333160eb34,0,"[3fcd7cdc-20e6-4ea3-a41c-db126fcc5cfe, bf3a11c...",Fur Visualisation for Computer Game Engines an...,international conference on computer vision an...,2014


In [35]:
df.head(100)

,abstract,authors,id,n_citation,references,title,venue,year
0,The purpose of this study is to develop a lear...,"[Makoto Satoh, Ryo Muramatsu, Mizue Kayama, Ka...",00127ee2-cb05-48ce-bc49-9de556b93346,0,"[51c7e02e-f5ed-431a-8cf5-f761f266d4be, 69b625b...",Preliminary Design of a Network Protocol Learn...,international conference on human-computer int...,2013
1,This paper describes the design and implementa...,"[Gareth Beale, Graeme Earl]",001c58d3-26ad-46b3-ab3a-c1e557d16821,50,"[10482dd3-4642-4193-842f-85f3b70fcf65, 3133714...",A methodology for the physically accurate visu...,visual analytics science and technology,2011
2,This article applied GARCH model instead AR or...,"[Altaf Hossain, Faisal Zaman, Mohammed Nasser,...",001c8744-73c4-4b04-9364-22d31a10dbf1,50,"[2d84c0f2-e656-4ce7-b018-90eda1c132fe, a083a1b...","Comparison of GARCH, Neural Network and Suppor...",pattern recognition and machine intelligence,2009
3,NaN,"[Jea-Bum Park, Byungmok Kim, Jian Shen, Sun-Yo...",00338203-9eb3-40c5-9f31-cbac73a519ec,0,"[8c78e4b0-632b-4293-b491-85b1976675e6, 9cdc54f...",Development of Remote Monitoring and Control D...,,2011
4,NaN,"[Giovanna Guerrini, Isabella Merlo]",0040b022-1472-4f70-a753-74832df65266,2,NaN,Reasonig about Set-Oriented Methods in Object ...,,1998
5,NaN,"[Rafael Álvarez, Leandro Tortosa, José-Francis...",005ce28f-ed77-4e97-afdc-a296137186a1,0,NaN,COMPARING GNG3D AND QUADRIC ERROR METRICS METH...,international conference on computer graphics ...,2009
6,NaN,"[Jovan Dj. Golic, Guglielmo Morgari]",00638a94-23bf-4fa6-b5ce-40d799c65da7,2,NaN,Vectorial fast correlation attacks.,,2004
7,NaN,"[Guzin Ulutas, Mustafa Ulutas, Vasif V. Nabiyev]",00701b05-684f-45f9-b281-425abfec482c,0,"[5626736c-e434-4e2d-8405-54940fab88ab, 8e87e87...",Improved Secret Image Sharing Method By Encodi...,international symposium on computer and inform...,2011
8,NaN,"[Pranay Chaudhuri, Hussein Thompson]",00745041-3636-4d18-bbec-783c4278c40d,0,NaN,A Self-Stabilizing Algorithm for Finding the C...,parallel and distributed processing techniques...,2003
9,NaN,"[Dominik Szajerman, Adam Jurczyński]",00964544-cbe2-4da9-bb5a-03333160eb34,0,"[3fcd7cdc-20e6-4ea3-a41c-db126fcc5cfe, bf3a11c...",Fur Visualisation for Computer Game Engines an...,international conference on computer vision an...,2014


In [43]:
df.isna().sum()

abstract      246354
authors            0
id                 0
n_citation         0
references    138520
title              0
venue              0
year               0
dtype: int64

In [37]:
df.describe()

,n_citation,year
count,1000000.000000,1000000.000000
mean,30.875730,2006.365124
std,137.942473,7.833833
min,0.000000,1937.000000
25%,1.000000,2003.000000
50%,9.000000,2008.000000
75%,50.000000,2012.000000
max,73362.000000,2017.000000


In [38]:
df.year.unique()

array([2013, 2011, 2009, 1998, 2004, 2003, 2014, 1995, 2002, 1991, 1979,
       2008, 2007, 2005, 2012, 2010, 2000, 2006, 1986, 1988, 2001, 1989,
       1994, 1981, 2015, 1999, 1997, 1992, 1990, 1996, 1993, 1985, 1987,
       1982, 1984, 2016, 1969, 2017, 1974, 1970, 1977, 1975, 1983, 1972,
       1973, 1959, 1980, 1976, 1978, 1968, 1971, 1962, 1966, 1963, 1960,
       1967, 1961, 1954, 1965, 1964, 1937, 1958, 1956, 1953, 1952, 1938,
       1957, 1955, 1944, 1948, 1946, 1941, 1951, 1939, 1949, 1942, 1950,
       1940])

In [44]:
venue_dict = {}
idx = 0
for index, row in df.iterrows():
    if row['venue'] == '':
        continue
    num_papers = 1
    num_citations = int(row['n_citation'])
    key = str(row['year']) + "::" + row['venue']
    if key in venue_dict:
        num_papers = int(venue_dict[key].split("::")[0]) + 1
        num_citations = int(venue_dict[key].split("::")[1]) + int(row['n_citation'])
    venue_dict[key] = str(num_papers) + "::" + str(num_citations)
#     print(key)
#     print(str(num_papers) + "::" + str(num_citations))
#     idx+=1
#     if idx>10:
#         break

In [45]:
len(venue_dict.keys())

39302

In [49]:
with open('./data/venues.csv', 'w') as csvfile:
    filewriter = csv.writer(csvfile, delimiter=',', quotechar='|', quoting=csv.QUOTE_MINIMAL)
    filewriter.writerow(['venue_id', 'venue_name', 'year', 'num_papers', 'avg_citations'])
    venue_id = 1
    for key, value in venue_dict.items():
        venue_name = str(key.split("::")[1])
        year = str(key.split("::")[0])
        num_papers = str(venue_dict[key].split("::")[0])
        avg_citations = float(venue_dict[key].split("::")[1])/int(venue_dict[key].split("::")[0])
        filewriter.writerow([str(venue_id), venue_name, year, num_papers, avg_citations])
        venue_id+=1